In [42]:
import gurobipy as gp
from gurobipy import GRB

In [43]:
# Create a new model
model = gp.Model("Courier_Assignment")

In [44]:
# Data
T = [1, 2, 3]  # Periods
O_t = {1: [1, 2, 3], 2: [4, 5, 6], 3: [7, 8, 9, 10]}  # Orders per period
C_t = {1: [1, 2, 3], 2: [4, 5], 3: [6, 7, 8]}  # Couriers per period

# Cost parameters (example values)
waiting_cost = {
    (1, 1): -2,
    (1, 2): -1,
    (1, 3): -3,
    (2, 1): -2,
    (2, 2): -1,
    (2, 3): -3,
    (3, 1): -2,
    (3, 2): -1,
    (3, 3): -3,
    (4, 4): -2,
    (4, 5): -1,
    (5, 4): -2,
    (5, 5): -1,
    (6, 4): -2,
    (6, 5): -1,
    (7, 6): -2,
    (7, 7): -1,
    (7, 8): -3,
    (8, 6): -2,
    (8, 7): -1,
    (8, 8): -3,
    (9, 6): -2,
    (9, 7): -1,
    (9, 8): -3,
    (10, 6): -2,
    (10, 7): -1,
    (10, 8): -3,
}

lateness_cost = {
    (1, 1): 5,
    (1, 2): 4,
    (1, 3): 6,
    (2, 1): 5,
    (2, 2): 4,
    (2, 3): 6,
    (3, 1): 5,
    (3, 2): 4,
    (3, 3): 6,
    (4, 4): 5,
    (4, 5): 4,
    (5, 4): 5,
    (5, 5): 4,
    (6, 4): 5,
    (6, 5): 4,
    (7, 6): 5,
    (7, 7): 4,
    (7, 8): 6,
    (8, 6): 5,
    (8, 7): 4,
    (8, 8): 6,
    (9, 6): 5,
    (9, 7): 4,
    (9, 8): 6,
    (10, 6): 5,
    (10, 7): 4,
    (10, 8): 6,
}

# Service capacity of couriers
u = {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3}

# Maximum workable time length of couriers
l = {1: 10, 2: 12, 3: 15, 4: 10, 5: 12, 6: 15, 7: 10, 8: 12}

# Time consumption per order for couriers
p = {1: 2, 2: 2.5, 3: 3, 4: 2.5, 5: 2.5, 6: 3.5, 7: 2.0, 8: 2.0}

# Cost of assigning order i to courier j
c = {
    (i, j): waiting_cost.get((i, j), 0) + lateness_cost.get((i, j), 0)
    for i in range(1, 11)
    for j in range(1, 9)
}

In [45]:
# Decision variables
x = model.addVars(
    [(i, j, t) for t in T for i in O_t[t] for j in C_t[t]], vtype=GRB.BINARY, name="x"
)

In [46]:
# Objective function
model.setObjective(
    gp.quicksum(c[(i, j)] * x[i, j, t] for t in T for i in O_t[t] for j in C_t[t]),
    GRB.MINIMIZE,
)

In [47]:
# Constraints
# Each order must be assigned to exactly one courier
model.addConstrs(
    (gp.quicksum(x[i, j, t] for j in C_t[t]) == 1 for t in T for i in O_t[t]),
    name="order_assignment",
)

# The number of orders assigned to one courier cannot exceed their service capacity
model.addConstrs(
    (gp.quicksum(x[i, j, t] for i in O_t[t]) <= u[j] for t in T for j in C_t[t]),
    name="service_capacity",
)

# The maximum allowable work time cannot be violated
model.addConstrs(
    (p[j] * gp.quicksum(x[i, j, t] for i in O_t[t]) <= l[j] for t in T for j in C_t[t]),
    name="work_time",
)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 7): <gurobi.Constr *Awaiting Model Update*>,
 (3, 8): <gurobi.Constr *Awaiting Model Update*>}

In [48]:
# Optimize model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x1a139644
Variable types: 0 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 30.0000000
Presolve removed 26 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+01, best bound 3.000000000000e+01, gap 0.0000%


In [49]:
# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print("Total cost:", model.ObjVal)
    print("\nOptimal assignment of orders to couriers:")
    for t in T:
        for i in O_t[t]:
            for j in C_t[t]:
                if x[i, j, t].x > 0.5:
                    print(f"Order {i} in period {t} is assigned to courier {j}")
else:
    print("No optimal solution found")

Optimal solution found:
Total cost: 30.0

Optimal assignment of orders to couriers:
Order 1 in period 1 is assigned to courier 1
Order 2 in period 1 is assigned to courier 1
Order 3 in period 1 is assigned to courier 1
Order 4 in period 2 is assigned to courier 5
Order 5 in period 2 is assigned to courier 5
Order 6 in period 2 is assigned to courier 5
Order 7 in period 3 is assigned to courier 7
Order 8 in period 3 is assigned to courier 6
Order 9 in period 3 is assigned to courier 6
Order 10 in period 3 is assigned to courier 6
